In [19]:
#from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import numpy as np
import sys
import tensorflow as tf

import os, re, glob
from sklearn.model_selection import train_test_split
import cv2 

#cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
groups_folder_path = r'C:\Users\ailab\PycharmProjects\face\Bulk-Bing-Image-downloader-master\AF'

#categories = ["0", "1"]
#num_classes = len(categories)

image_w = 128
image_h = 128
  
X = []
Y = []

#
image_dir = groups_folder_path + '/'
idex = 0   
label = []
for top, dir, f in os.walk(image_dir):
    print(len(f))
    for filename in f:           
        if 'angry' in filename :
            print("a ", filename)
            label = 0
        elif 'neutral' in filename:  
            print("n ", filename)
            label = 1   
        #elif 'happy' in filename:  
            #print("h ", filename)
            #label = 2 
        #elif 'fearful' in filename:  
            #print("f ", filename)
            #label = 3
        elif 'surprising' in filename:
            print("su ", filename)
            label = 4
        elif 'sad' in filename:
            print("s ", filename)
            label = 5
            
        #print(image_dir+filename)
        img = cv2.imread(image_dir+filename)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(gray_img, 1.03, 5)

        if type(faces) == tuple:
            continue
        else:
            if faces.shape[0] == 1:
                for (x,y,w,h) in faces:
                    cropped = gray_img[y:y + h, x:x + w]
                re_gray = cv2.resize(cropped, None, fx=image_w/cropped.shape[0], fy=image_h/cropped.shape[1])            
                #image.resize???              
        X.append(re_gray)
        Y.append(label)
        #cv2.imshow('image', re_gray)
        #cv2.waitKey(0)
                    
X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
print(X_train)
print(Y_train)

#cv2.imshow('image', X_train[0])
#cv2.waitKey(0)
#xy = (X_train, X_test, Y_train, Y_test)
#cv2.destroyAllWindows()


ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [4]:
#(X_train, Y_class_train), (X_test, Y_class_test) = mnist.load_data()
#X_train, X_test, Y_train, Y_test 

#총 100개의 원소가 들어있는 배열 x에 대해서 x.reshape(-1, 정수) 를 해주면 
#'열(column)' 차원의 '정수'에 따라서 100개의 원소가 빠짐없이 배치될 수 있도록 
#'-1'이 들어가 있는 '행(row)' 의 개수가 가변적으로 정해짐
X_train = X_train.reshape(-1, 128, 128,1).astype('float64') / 255
X_test = X_test.reshape(-1,128,128,1).astype('float64') / 255

#만약 감정을 분노와 무표정에 더 많이 추가한다면 카테고리 2에서 n으로 수정하면 됩니다.
Y_train = np_utils.to_categorical(Y_train, 2)
Y_test = np_utils.to_categorical(Y_test, 2)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), input_shape = (128, 128, 1), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', 
             optimizer = 'adam',
             metrics = ['accuracy'])
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', verbose = 1, save_best_only = True)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience=10)

history = model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 30, batch_size = 50, verbose = 0,
                   callbacks = [early_stopping_callback, checkpointer])



Epoch 00001: val_loss improved from inf to 0.76080, saving model to ./model/01-0.7608.hdf5

Epoch 00002: val_loss improved from 0.76080 to 0.55224, saving model to ./model/02-0.5522.hdf5

Epoch 00003: val_loss improved from 0.55224 to 0.42229, saving model to ./model/03-0.4223.hdf5

Epoch 00004: val_loss improved from 0.42229 to 0.31668, saving model to ./model/04-0.3167.hdf5

Epoch 00005: val_loss improved from 0.31668 to 0.30421, saving model to ./model/05-0.3042.hdf5

Epoch 00006: val_loss improved from 0.30421 to 0.24142, saving model to ./model/06-0.2414.hdf5

Epoch 00007: val_loss improved from 0.24142 to 0.20743, saving model to ./model/07-0.2074.hdf5

Epoch 00008: val_loss did not improve

Epoch 00009: val_loss did not improve

Epoch 00010: val_loss improved from 0.20743 to 0.19804, saving model to ./model/10-0.1980.hdf5

Epoch 00011: val_loss did not improve

Epoch 00012: val_loss did not improve

Epoch 00013: val_loss did not improve

Epoch 00014: val_loss improved from 0.19